In [2]:
import os

import glob
import copy
import numpy as np
import scipy
import scipy.optimize
import scipy.interpolate
import astropy.table
import astropy.io.fits
import astropy.units

import matplotlib.pyplot as plt
import matplotlib as mpl


import lezargus

import pycpd

%matplotlib Qt

ToDoError - Uncertainty values on integrations need to be done.
ToDoError - Uncertainty values on integrations need to be done.
ToDoError - Uncertainty values on integrations need to be done.
ToDoError - Uncertainty values on integrations need to be done.
ToDoError - Uncertainty values on integrations need to be done.
ToDoError - Uncertainty values on integrations need to be done.
ToDoError - Uncertainty values on integrations need to be done.
ToDoError - Uncertainty values on integrations need to be done.
ToDoError - Uncertainty values on integrations need to be done.
ToDoError - Uncertainty values on integrations need to be done.
ToDoError - Uncertainty values on integrations need to be done.
ToDoError - Uncertainty values on integrations need to be done.
ToDoError - Uncertainty values on integrations need to be done.
ToDoError - Uncertainty values on integrations need to be done.
ToDoError - Uncertainty values on integrations need to be done.
ToDoError - Uncertainty values on integr

In [3]:
lezargus.initialize.initialize_logging_outputs()
# Making sure that the output directory exists.
os.makedirs("./products/spectre_dispersion/", exist_ok=True)

# Slice Pattern File

In [4]:
# Reading the base file, and saving the new file.
slice_pattern_filename = "./base/spectre_dispersion/spectre_slice_pattern.txt"
pattern_data = np.genfromtxt(slice_pattern_filename, comments="#").T

pattern_slice_index = pattern_data[0]
pattern_slice_x = pattern_data[1]
pattern_slice_y = pattern_data[2]

# The pattern values in the file are given in millimeters, we convert it to
# meters per convention.
pattern_slice_x = pattern_slice_x / 1000
pattern_slice_y = pattern_slice_y / 1000

pattern_index_column = astropy.table.Column(
    pattern_slice_index, name="index", unit="", dtype=int
)
pattern_x_column = astropy.table.Column(
    pattern_slice_x, name="x", unit="m", dtype=float
)
pattern_y_column = astropy.table.Column(
    pattern_slice_y, name="y", unit="m", dtype=float
)

table_columns = [pattern_index_column, pattern_x_column, pattern_y_column]
pattern_table = astropy.table.Table(table_columns)
# Too much precision makes unnessary large file sizes.
pattern_filename = f"./products/spectre_dispersion/spectre_slice_pattern.dat"
pattern_table.write(
    pattern_filename,
    format="ascii.mrt",
    formats={"index": "%4d"}
    | {keydex: "%.3e" for keydex in pattern_table.keys()},
    overwrite=True,
)
pattern_pairs = np.array([pattern_slice_x, pattern_slice_y]).T

# Disperstion Table File

In [5]:
# Reading the template file which we will use to generate all of the
# other points.
template_dispersion_filename = (
    "./base/spectre_dispersion/spectre_slice_dispersion.txt"
)
template_dispersion_table = astropy.table.Table.read(
    template_dispersion_filename,
    format="ascii.basic",
    delimiter="|",
    comment="#",
)
# Convert to meters.
template_dispersion_table["LeftX"] /= 1000
template_dispersion_table["LeftY"] /= 1000
template_dispersion_table["CenterX"] /= 1000
template_dispersion_table["CenterY"] /= 1000
template_dispersion_table["RightX"] /= 1000
template_dispersion_table["RightY"] /= 1000

In [6]:
def get_points_dispersion_table(
    channel: str, wavelength: float, location: str
) -> np.ndarray:
    """Get the points from the dispersion table.

    Parameters
    ----------
    channel : str
        The channel: VIS, NIR, MIR.
    wavelength : float
        The wavelength desired, but it must exist.
    location : str
        The location of the points within the slice, either "Left", "Center",
        or "Right".

    Returns
    -------
    points : ndarray
        The points.
    """
    # We filter the main table.
    filter_channel = template_dispersion_table["Channel"] == channel
    filter_wavelength = template_dispersion_table["Wavelength"] == wavelength
    filter_index = filter_channel & filter_wavelength
    filtered_table = template_dispersion_table[filter_index]

    point_x = np.array(filtered_table[f"{location}X"])
    point_y = np.array(filtered_table[f"{location}Y"])
    points = np.array([point_x, point_y]).T
    return points


def transform_pattern(pattern: np.ndarray, anchors: np.ndarray) -> np.ndarray:
    """Register/transform the provided point pattern to fit the anchor points.

    Parameters
    ----------
    pattern : ndarray
        The point pattern that we are registering. For most cases, this is the
        36 slice point pattern.
    anchors : ndarray
        The anchor points we are fitting the pattern to. For most cases, this
        is the wavelength dispersion slice points.

    Returns
    -------
    new_pattern : ndarray
        The point pattern after being transformed to fit the anchor points.
    """
    # The order of the pattern and anchors is important.
    transformation_mode = "affine"
    if transformation_mode == "rigid":
        registration = pycpd.RigidRegistration(X=anchors, Y=pattern)
        new_pattern, (_scale, _rotation, _translation) = registration.register()
    elif transformation_mode == "affine":
        registration = pycpd.AffineRegistration(X=anchors, Y=pattern)
        new_pattern, (_matrix, _translation) = registration.register()
    else:
        raise ValueError("Unknown transformation mode.")
    # As the registration above does the transformations for us, we can ignore
    # them.
    return new_pattern


def channel_long_name(short_name: str):
    """Just, write out the channel name.

    Parameters
    ----------
    short_name : str
        The short name: VIS, NIR, MIR

    Returns
    -------
    long_name : str
        The long name.
    """
    short_name = short_name.upper()
    channel_translation = {"VIS": "visible", "NIR": "nearir", "MIR": "midir"}
    return channel_translation[short_name]

In [7]:
# The unique combinations available for the dispersion file.
dispersion_combinations = (
    # Channel, wavelength
    # Visible...
    ["VIS", 0.310],
    ["VIS", 0.370],
    ["VIS", 0.440],
    ["VIS", 0.510],
    ["VIS", 0.580],
    ["VIS", 0.650],
    ["VIS", 0.720],
    ["VIS", 0.790],
    ["VIS", 0.860],
    ["VIS", 0.930],
    ["VIS", 1.000],
    # near-ir...
    ["NIR", 0.700],
    ["NIR", 0.900],
    ["NIR", 1.100],
    ["NIR", 1.300],
    ["NIR", 1.500],
    ["NIR", 1.700],
    ["NIR", 1.900],
    ["NIR", 2.100],
    ["NIR", 2.300],
    ["NIR", 2.500],
    ["NIR", 2.700],
    # mid-ir....
    ["MIR", 2.000],
    ["MIR", 2.300],
    ["MIR", 2.600],
    ["MIR", 2.900],
    ["MIR", 3.200],
    ["MIR", 3.500],
    ["MIR", 3.800],
    ["MIR", 4.100],
    ["MIR", 4.400],
    ["MIR", 4.700],
)
# The anchor points do not provide any indication of height, so we calculate
# the height of the slice assuming that the height to width ratio is always the
# same. Height / Width.
height_width_ratio = 2 / 72


# Color map functionality for debugging plots.
def color_function(wavelength: float):
    # Parameters
    min_wave = 0.40
    max_wave = 4.20

    # We use energy as map.
    min_energy = 1 / max_wave
    max_energy = 1 / min_wave
    energy = 1 / wavelength

    color_map = copy.deepcopy(mpl.colormaps["jet_r"])
    normalize = mpl.colors.Normalize(
        vmin=min_energy, vmax=max_energy, clip=True
    )
    color = color_map(normalize(energy))
    return color

### Doing the work...

In [8]:
# Table rows.
full_dispersion_rows = []

plt.figure()
for channeldex, wavedex in dispersion_combinations:
    # Left
    left_anchor = get_points_dispersion_table(
        channel=channeldex, wavelength=wavedex, location="Left"
    )
    left_pattern = transform_pattern(pattern=pattern_pairs, anchors=left_anchor)
    # Center
    center_anchor = get_points_dispersion_table(
        channel=channeldex, wavelength=wavedex, location="Center"
    )
    center_pattern = transform_pattern(
        pattern=pattern_pairs, anchors=center_anchor
    )
    # Right
    right_anchor = get_points_dispersion_table(
        channel=channeldex, wavelength=wavedex, location="Right"
    )
    right_pattern = transform_pattern(
        pattern=pattern_pairs, anchors=right_anchor
    )

    # We use the width to figure out the top and bottom points based on the
    # ratio.
    left_x_points = left_pattern.T[0]
    right_x_points = right_pattern.T[0]
    widths = np.abs(right_x_points - left_x_points)
    heights = height_width_ratio * widths
    half_height = heights / 2

    # We assume the top and bottom points are directly offset in Y by the half
    # height from the center.
    top_pattern = np.array(
        [
            [xdex, ydex + heightdex]
            for (xdex, ydex), heightdex in zip(center_pattern, half_height)
        ]
    )
    bot_pattern = np.array(
        [
            [xdex, ydex - heightdex]
            for (xdex, ydex), heightdex in zip(center_pattern, half_height)
        ]
    )

    # Debug plot.
    if True:
        # The found pattern points, in reverse order of importance.
        plt.scatter(*bot_pattern.T, marker=".", color=color_function(wavedex))
        plt.scatter(*top_pattern.T, marker=".", color=color_function(wavedex))
        plt.scatter(*right_pattern.T, marker=".", color=color_function(wavedex))
        plt.scatter(*left_pattern.T, marker=".", color=color_function(wavedex))
        plt.scatter(
            *center_pattern.T, marker=".", color=color_function(wavedex)
        )
        # And the anchors to compare it to.
        plt.scatter(*left_anchor.T, marker="2", color="pink")
        plt.scatter(*center_anchor.T, marker="*", color="pink")
        plt.scatter(*right_anchor.T, marker="2", color="pink")
        # Millimeter coordinate tick marks are easier.
        plt.show()

    for sliceindex, (cdex, tdex, bdex, ldex, rdex) in enumerate(
        zip(
            center_pattern,
            top_pattern,
            bot_pattern,
            left_pattern,
            right_pattern,
        )
    ):
        # Convert wavelength to meters.
        wavedex_m = wavedex * 1e-6

        row_dict = {
            "channel": channel_long_name(short_name=channeldex),
            "slice": sliceindex + 1,
            "wavelength": wavedex_m * astropy.units.Unit("m"),
            "center_x": cdex[0] * astropy.units.Unit("m"),
            "center_y": cdex[1] * astropy.units.Unit("m"),
            "top_x": tdex[0] * astropy.units.Unit("m"),
            "top_y": tdex[1] * astropy.units.Unit("m"),
            "bottom_x": bdex[0] * astropy.units.Unit("m"),
            "bottom_y": bdex[1] * astropy.units.Unit("m"),
            "left_x": ldex[0] * astropy.units.Unit("m"),
            "left_y": ldex[1] * astropy.units.Unit("m"),
            "right_x": rdex[0] * astropy.units.Unit("m"),
            "right_y": rdex[1] * astropy.units.Unit("m"),
        }
        full_dispersion_rows.append(row_dict)

plt.show()

In [9]:
# Creating and saving the table.
full_dispersion_table = astropy.table.Table(rows=full_dispersion_rows)

# Types
table_type_formats = {
    "channel": str,
    "slice": "{:d}",
    "wavelength": "{: .3e}",
    "center_x": "{: .9f}",
    "center_y": "{: .9f}",
    "top_x": "{: .9f}",
    "top_y": "{: .9f}",
    "bottom_x": "{: .9f}",
    "bottom_y": "{: .9f}",
    "left_x": "{: .9f}",
    "left_y": "{: .9f}",
    "right_x": "{: .9f}",
    "right_y": "{: .9f}",
}

full_dispersion_filename = (
    f"./products/spectre_dispersion/spectre_slice_dispersion.dat"
)
full_dispersion_table.write(
    full_dispersion_filename,
    format="ascii.mrt",
    formats=table_type_formats,
    overwrite=True,
)

In [10]:
test_dispersion_table = astropy.table.Table.read(
    full_dispersion_filename,
    format="ascii.mrt",
)

### Manual Interpolation Via Fine Grid Wavelength

Visible

In [11]:
# The fine and the corse grids are based on a single center slice.
# Though labeled as slice 5 in the file, this is the slice 5 of 9.
visible_fine_wave = np.array(
    [0.31, 0.37, 0.44, 0.51, 0.58, 0.65, 0.72, 0.79, 0.86, 0.93, 1.00]
)
raw_visible_fine_x = np.array(
    [
        -0.646,
        -0.645,
        -0.644,
        -0.643,
        -0.643,
        -0.643,
        -0.643,
        -0.642,
        -0.642,
        -0.642,
        -0.642,
    ]
)
visible_fine_x = np.array(
    [
        -0.646,
        -0.645,
        -0.644,
        -0.6435,
        -0.643,
        -0.643,
        -0.643,
        -0.6425,
        -0.642,
        -0.642,
        -0.642,
    ]
)
visible_fine_y = np.array(
    [
        10.94,
        6.015,
        2.804,
        0.864,
        -0.423,
        -1.343,
        -2.040,
        -2.597,
        -3.062,
        -3.466,
        -3.829,
    ]
)

visible_corse_wave = np.array([0.400, 0.500, 0.850])
visible_corse_x = np.array([-0.64, -0.64, -0.64])
visible_corse_y = np.array([4.42, 1.09, -3.00])

# Making smooth functions out of them.
interp_x = scipy.interpolate.make_smoothing_spline(
    visible_fine_wave, visible_fine_x, lam=0.0006
)
interp_y = scipy.interpolate.Akima1DInterpolator(
    visible_fine_wave, visible_fine_y
)

# Now, we need to map it to a new off slice.
# Change here!
visible_off_wave = np.array([0.400, 0.500, 0.850])
visible_off_x = np.array([-14.77, -14.78, -14.79])
visible_off_y = np.array([-2.94, -6.25, -10.35])


def linear_transform_x(w, m, b):
    return m * interp_x(w) + b


def linear_transform_y(w, m, b):
    return m * interp_y(w) + b


# Fitting the curves to the new points.
param_x, __ = scipy.optimize.curve_fit(
    linear_transform_x, visible_off_wave, visible_off_x
)
param_y, __ = scipy.optimize.curve_fit(
    linear_transform_y, visible_off_wave, visible_off_y
)

plot_wave = np.linspace(0.31, 1.0, 1000)

plt.figure()
plt.title("Visible X Coordinate")
plt.scatter(visible_off_wave, visible_off_x, color="orange", marker="h")
plt.scatter(
    visible_corse_wave, param_x[0] * visible_corse_x + param_x[1], marker="h"
)
plt.scatter(visible_fine_wave, param_x[0] * visible_fine_x + param_x[1])
plt.scatter(
    visible_fine_wave, param_x[0] * raw_visible_fine_x + param_x[1], marker="*"
)
plt.plot(plot_wave, linear_transform_x(plot_wave, *param_x))
plt.xlabel("Wavelength [um]")
plt.ylabel("X Location [mm]")
plt.show()


plt.figure()
plt.title("Visible Y Coordinate")
plt.scatter(visible_off_wave, visible_off_y, color="orange", marker="h")
plt.scatter(
    visible_corse_wave, param_y[0] * visible_corse_y + param_y[1], marker="h"
)
plt.scatter(visible_fine_wave, param_y[0] * visible_fine_y + param_y[1])
plt.plot(plot_wave, linear_transform_y(plot_wave, *param_y))
plt.xlabel("Wavelength [um]")
plt.ylabel("Y Location [mm]")
plt.show()

# Reading out the new set to copy over.
print("New Set")
for wavedex in visible_fine_wave:
    new_x = linear_transform_x(wavedex, *param_x)
    new_y = linear_transform_y(wavedex, *param_y)
    print(f"{wavedex:6.3f}    {new_x:6.3f}    {new_y:6.3f}")

New Set
 0.310    -14.761     3.589
 0.370    -14.768    -1.333
 0.440    -14.774    -4.543
 0.510    -14.779    -6.482
 0.580    -14.783    -7.768
 0.650    -14.785    -8.688
 0.720    -14.787    -9.384
 0.790    -14.789    -9.941
 0.860    -14.791    -10.406
 0.930    -14.792    -10.810
 1.000    -14.793    -11.173


NIR

In [12]:
# The fine and the corse grids are based on a single center slice.
# Though labeled as slice 5 in the file, this is the slice 5 of 9.
nearir_fine_wave = np.array(
    [0.7, 0.9, 1.1, 1.3, 1.5, 1.7, 1.9, 2.1, 2.3, 2.5, 2.7]
)
raw_nearir_fine_x = np.array(
    [
        -0.778,
        -0.773,
        -0.770,
        -0.767,
        -0.764,
        -0.762,
        -0.758,
        -0.755,
        -0.751,
        -0.747,
        -0.742,
    ]
)
nearir_fine_x = np.array(
    [
        -0.778,
        -0.773,
        -0.770,
        -0.767,
        -0.764,
        -0.762,
        -0.758,
        -0.755,
        -0.751,
        -0.747,
        -0.742,
    ]
)
nearir_fine_y = np.array(
    [
        6.885,
        3.643,
        1.901,
        0.693,
        -0.304,
        -1.227,
        -2.145,
        -3.096,
        -4.105,
        -5.191,
        -6.367,
    ]
)

nearir_corse_wave = np.array([0.85, 1.25, 2.40])
nearir_corse_x = np.array([15.60, 15.61, 15.61])
nearir_corse_y = np.array([4.24, 0.97, -4.64])

# Making smooth functions out of them.
interp_x = scipy.interpolate.make_smoothing_spline(
    nearir_fine_wave, nearir_fine_x, lam=0.0006
)
interp_y = scipy.interpolate.Akima1DInterpolator(
    nearir_fine_wave, nearir_fine_y
)

# Now, we need to map it to a new off slice.
nearir_off_wave = np.array([0.85, 1.25, 2.40])
nearir_off_x = np.array([0.12, 0.13, 0.16])
nearir_off_y = np.array([-4.99, -8.25, -13.83])


def linear_transform_x(w, m, b):
    return m * interp_x(w) + b


def linear_transform_y(w, m, b):
    return m * interp_y(w) + b


# Fitting the curves to the new points.
param_x, __ = scipy.optimize.curve_fit(
    linear_transform_x, nearir_off_wave, nearir_off_x
)
param_y, __ = scipy.optimize.curve_fit(
    linear_transform_y, nearir_off_wave, nearir_off_y
)

plot_wave = np.linspace(0.70, 2.70, 1000)

plt.figure()
plt.title("Nearir X Coordinate")
plt.scatter(nearir_off_wave, nearir_off_x, color="orange", marker="h")
# plt.scatter(
#    nearir_corse_wave, param_x[0] * nearir_corse_x + param_x[1], marker="h"
# )
plt.scatter(nearir_fine_wave, param_x[0] * nearir_fine_x + param_x[1])
plt.scatter(
    nearir_fine_wave, param_x[0] * raw_nearir_fine_x + param_x[1], marker="*"
)
plt.plot(plot_wave, linear_transform_x(plot_wave, *param_x))
plt.xlabel("Wavelength [um]")
plt.ylabel("X Location [mm]")
plt.show()


plt.figure()
plt.title("Nearir Y Coordinate")
plt.scatter(nearir_off_wave, nearir_off_y, color="orange", marker="h")
plt.scatter(
    nearir_corse_wave, param_y[0] * nearir_corse_y + param_y[1], marker="h"
)
plt.scatter(nearir_fine_wave, param_y[0] * nearir_fine_y + param_y[1])
plt.plot(plot_wave, linear_transform_y(plot_wave, *param_y))
plt.xlabel("Wavelength [um]")
plt.ylabel("Y Location [mm]")
plt.show()

# Reading out the new set to copy over.
print("New Set")
for wavedex in nearir_fine_wave:
    new_x = linear_transform_x(wavedex, *param_x)
    new_y = linear_transform_y(wavedex, *param_y)
    print(f"{wavedex:6.3f}    {new_x:6.3f}    {new_y:6.3f}")

New Set
 0.700     0.114    -2.379
 0.900     0.121    -5.600
 1.100     0.127    -7.331
 1.300     0.131    -8.531
 1.500     0.136    -9.521
 1.700     0.140    -10.438
 1.900     0.145    -11.350
 2.100     0.151    -12.295
 2.300     0.157    -13.298
 2.500     0.163    -14.377
 2.700     0.171    -15.545


Midir

In [15]:
# The fine and the corse grids are based on a single center slice.
# Though labeled as slice 5 in the file, this is the slice 5 of 9.
midir_fine_wave = np.array([2.0, 2.3, 2.6, 2.9, 3.2, 3.5, 3.8, 4.1, 4.4, 4.7])
raw_midir_fine_x = np.array(
    [
        -0.806,
        -0.798,
        -0.791,
        -0.783,
        -0.773,
        -0.763,
        -0.752,
        -0.740,
        -0.727,
        -0.712,
    ]
)
midir_fine_x = np.array(
    [
        -0.806,
        -0.798,
        -0.791,
        -0.783,
        -0.773,
        -0.763,
        -0.752,
        -0.740,
        -0.727,
        -0.712,
    ]
)
midir_fine_y = np.array(
    [6.060, 5.158, 4.140, 2.991, 1.703, 0.269, -1.320, -3.073, -5.003, -7.121]
)

midir_corse_wave = np.array([2.4, 3.2, 4.2])
midir_corse_x = np.array([-0.80, -0.77, -0.74])
midir_corse_y = np.array([4.83, 1.70, -3.70])


# Making smooth functions out of them.
interp_x = scipy.interpolate.make_smoothing_spline(
    midir_fine_wave, midir_fine_x, lam=0.0006
)
interp_y = scipy.interpolate.Akima1DInterpolator(midir_fine_wave, midir_fine_y)

# Now, we need to map it to a new off slice.
midir_off_wave = np.array([2.4, 3.2, 4.2])
midir_off_x = np.array([-17.87, -17.83, -17.78])
midir_off_y = np.array([-4.28, -7.40, -12.77])


def linear_transform_x(w, m, b):
    return m * interp_x(w) + b


def linear_transform_y(w, m, b):
    return m * interp_y(w) + b


# Fitting the curves to the new points.
param_x, __ = scipy.optimize.curve_fit(
    linear_transform_x, midir_off_wave, midir_off_x
)
param_y, __ = scipy.optimize.curve_fit(
    linear_transform_y, midir_off_wave, midir_off_y
)

plot_wave = np.linspace(2.0, 4.7, 1000)

plt.figure()
plt.title("Midir X Coordinate")
plt.scatter(midir_off_wave, midir_off_x, color="orange", marker="h")
plt.scatter(
    midir_corse_wave, param_x[0] * midir_corse_x + param_x[1], marker="h"
)
plt.scatter(midir_fine_wave, param_x[0] * midir_fine_x + param_x[1])
plt.scatter(
    midir_fine_wave, param_x[0] * raw_midir_fine_x + param_x[1], marker="*"
)
plt.plot(plot_wave, linear_transform_x(plot_wave, *param_x))
plt.xlabel("Wavelength [um]")
plt.ylabel("X Location [mm]")
plt.show()


plt.figure()
plt.title("Midir Y Coordinate")
plt.scatter(midir_off_wave, midir_off_y, color="orange", marker="h")
plt.scatter(
    midir_corse_wave, param_y[0] * midir_corse_y + param_y[1], marker="h"
)
plt.scatter(midir_fine_wave, param_y[0] * midir_fine_y + param_y[1])
plt.plot(plot_wave, linear_transform_y(plot_wave, *param_y))
plt.xlabel("Wavelength [um]")
plt.ylabel("Y Location [mm]")
plt.show()

# Reading out the new set to copy over.
print("New Set")
for wavedex in midir_fine_wave:
    new_x = linear_transform_x(wavedex, *param_x)
    new_y = linear_transform_y(wavedex, *param_y)
    print(f"{wavedex:6.3f}    {new_x:6.3f}    {new_y:6.3f}")

New Set
 2.000    -17.883    -3.060
 2.300    -17.871    -3.958
 2.600    -17.861    -4.971
 2.900    -17.849    -6.115
 3.200    -17.834    -7.397
 3.500    -17.819    -8.824
 3.800    -17.803    -10.406
 4.100    -17.785    -12.151
 4.400    -17.765    -14.073
 4.700    -17.743    -16.181
